In [30]:
import re
import pickle
import pandas as pd
import numpy as np
from scipy import sparse
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from scipy import sparse
import scipy.stats as sts

from datetime import datetime, timedelta, timezone

from scipy.special import expit

## Продвинутое машинное обучение: 
### Домашнее задание 2
Второе домашнее задание — самое большое в курсе, в нём придётся и концептуально подумать о происходящем, и технические трудности тоже порешать. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 

Второе задание — это полноценный проект по анализу данных, начиная от анализа постановки задачи и заканчивая сравнением результатов разных моделей. Задача реальная и серьёзная, хотя тему я выбрал развлекательную: ***мы будем строить вероятностную рейтинг-систему для спортивного “Что? Где? Когда?” (ЧГК)*** .

Background: в спортивном “Что? Где? Когда?” соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно 36 или 45, иногда 60, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. Соответственно, нам нужно:

 - построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;
 - при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;
 - а что сильно упрощает задачу и переводит её в область домашних заданий на EM-алгоритм — это характер данных: начиная с какого-то момента, в базу результатов начали вносить все повопросные результаты команд, т.е. в данных будут записи вида “какая команда на какой вопрос правильно ответила”.

Я сделал за вас только первый шаг: выкачал через API сайта рейтинга ЧГК все нужные данные, чтобы сайт не прилёг под вашими многочисленными скрейперами. :) Полученные данные лежат в формате pickle вот здесь:
https://www.dropbox.com/s/s4qj0fpsn378m2i/chgk.zip 

#### Задание 1
Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.


In [2]:
with open("results.pkl", "rb") as handler:
    results = pickle.load(handler)

In [3]:
players_dictionary = dict()
with open("players.pkl", "rb") as handler:
    players = pickle.load(handler)
    players_dictionary = players
    players_df = pd.DataFrame(players)
    for player in players:
        players_dictionary[player] = players[player]["surname"] + " " + players[player]["name"]
    
players_df = players_df.transpose()
players_df.sample(5)

,id,name,patronymic,surname
150020,150020,Илларион,,Глущенко
86786,86786,Александр,Михайлович,Баженов
29693,29693,Елена,,Смирнова
139720,139720,Каролина,,Мурадян
207909,207909,Григорий,Вячеславович,Орлов


In [4]:
with open("tournaments.pkl", "rb") as handler:
    tournaments = pickle.load(handler)
    tournaments_df = pd.DataFrame(tournaments)

In [5]:
tournaments_df = tournaments_df.transpose()
tournaments_df.dateStart = pd.to_datetime(tournaments_df.dateStart)
tournaments_df.sample(5)

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
1060,1060,Чемпионат Одессы. Высшая лига,2000-05-21 00:00:00+04:00,2000-05-21T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/5,[],None,None
4140,4140,Чемпионат Москвы. Высшая лига,2016-12-17 00:00:00+03:00,2017-04-15T20:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/50,"[{'id': 5990, 'name': 'Андрей', 'patronymic': ...",None,"{'1': 12, '2': 12, '3': 12, '4': 12, '5': 12, ..."
5891,5891,Кубок малой родины,2020-02-29 12:00:00+03:00,2020-03-01T14:30:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 37362, 'name': 'Сергей', 'patronymic':...",None,"{'1': 12, '2': 12, '3': 12, '4': 12, '5': 12, ..."
3566,3566,Чемпионат Минска. Лига А. Тур второй.,2015-11-24 18:45:00+03:00,2015-11-24T21:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/49,"[{'id': 32004, 'name': 'Иван', 'patronymic': '...",None,"{'1': 18, '2': 18}"
5269,5269,Чемпионат МГУ. Высшая лига. Второй игровой день,2018-12-14 19:00:00+03:00,2018-12-14T22:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/52,"[{'id': 5990, 'name': 'Андрей', 'patronymic': ...",None,"{'1': 12, '2': 12, '3': 12}"


In [6]:
tournaments_train = tournaments_df[(tournaments_df['dateStart'] >= datetime(2019,1,1,tzinfo=timezone(offset=timedelta())))] 
tournaments_train = tournaments_train[(tournaments_train['dateStart'] < datetime(2020,1,1,tzinfo=timezone(offset=timedelta())))]

In [7]:
tournaments_test = tournaments_df[(tournaments_df['dateStart'] >= datetime(2020,1,1,tzinfo=timezone(offset=timedelta())))] 
tournaments_test = tournaments_test[(tournaments_test['dateStart'] < datetime(2021,1,1,tzinfo=timezone(offset=timedelta())))]

Проверка на наличие mask и состава команд

In [8]:
# train
idx_correct = []
for ids in tournaments_train['id']:
    if len(results[ids]) != 0 and 'mask' in results[ids][0].keys() and results[ids][0]['mask'] is not None:
        idx_correct.append(ids)

In [9]:
tournaments_train = tournaments_train.loc[idx_correct]
print(len(tournaments_train))
tournaments_train.head()

676


,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
4772,4772,Синхрон северных стран. Зимний выпуск,2019-01-05 19:00:00+03:00,2019-01-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-01-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4973,4973,Балтийский Берег. 3 игра,2019-01-25 19:05:00+03:00,2019-01-29T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-01-28T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4974,4974,Балтийский Берег. 4 игра,2019-03-01 19:05:00+03:00,2019-03-05T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-03-04T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4975,4975,Балтийский Берег. 5 игра,2019-04-05 19:05:00+03:00,2019-04-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-04-08T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4986,4986,ОВСЧ. 6 этап,2019-02-15 20:00:00+03:00,2019-02-19T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}"


In [10]:
# test
idx_correct = []
for ids in tournaments_test['id']:
    if len(results[ids]) != 0 and 'mask' in results[ids][0].keys() and results[ids][0]['mask'] is not None:
        idx_correct.append(ids)

In [11]:
tournaments_test = tournaments_test.loc[idx_correct]
print(len(tournaments_test))
tournaments_test.head()

171


,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
4957,4957,Синхрон Биркиркары,2020-02-21 00:00:00+03:00,2020-02-27T23:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 2421, 'name': 'Ася', 'patronymic': 'Се...",{'dateRequestsAllowedTo': '2020-02-27T18:00:00...,"{'1': 13, '2': 13, '3': 13}"
5414,5414,Синхрон северных стран,2020-01-03 19:00:00+03:00,2020-01-10T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2020-01-10T23:59:00...,"{'1': 12, '2': 12, '3': 12}"
5477,5477,Онлайн: Синхрон Урюбджирова,2020-04-18 19:00:00+03:00,2020-04-30T19:00:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 91324, 'name': 'Эрдни', 'patronymic': ...",{'dateRequestsAllowedTo': '2020-04-30T23:55:00...,"{'1': 12, '2': 12, '3': 12}"
5707,5707,Школьный Синхрон-lite. Выпуск 3.6,2020-02-01 00:05:00+03:00,2020-03-09T23:55:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2020-03-06T23:59:00...,"{'1': 12, '2': 12, '3': 12}"
5708,5708,(а)Синхрон-lite. Лига старта. Эпизод XII,2020-02-01 00:05:00+03:00,2020-03-09T23:55:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2020-03-06T23:59:00...,"{'1': 12, '2': 12, '3': 12}"


#### Задание 2
Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


***Решение***: 
- X - это марица (кол-во игроков + кол-во вопросов) на (ответ каждого игрока) и 1 на индексе игрок и номеру вопроса ( сумма по строке = 2)
- y - 1 или 0, 1 - если правильный ответ

In [12]:
dict_players = {}
count_answers = {}

In [13]:
len(dict_players)

0

In [14]:
y = []
idx_where_1_player = []
idx_where_1_answers = []
count_anwer = 0
count_answers_team = 0
count_answers_tour = 0
answers = {}
teams = []
answer_list = []
team_id = 0
for id_tour in tournaments_train['id']:
    count_max_q = 0 
    for team in results[id_tour]:
        count_answers_team = count_answers_tour
        list_players_team = []
        if team['mask'] is None:
            continue
        for player in team['teamMembers']:
            if player['player']['id'] in dict_players.keys():
                list_players_team.append(dict_players[player['player']['id']])
                count_answers[player['player']['id']] += len(team['mask'])
            else:
                list_players_team.append(len(dict_players))
                dict_players[player['player']['id']] = len(dict_players)
                count_answers[player['player']['id']] = 0
        if count_max_q < len(team['mask']):
            count_max_q = len(team['mask'])
        
        for anwer in team['mask']:
            for id_player in list_players_team:
                idx_where_1_player.append([count_anwer, id_player])
                idx_where_1_answers.append([count_anwer, count_answers_team ])
                count_anwer += 1
                teams.append(team_id)
                answer_list.append(count_answers_team)
                if anwer == '1':
                    y.append(1)
                else:
                    y.append(0)
            count_answers_team += 1
        team_id += 1
    for i in range(count_answers_tour,count_answers_tour+count_max_q):
        answers[i] = tournaments_train['name'][id_tour]
    count_answers_tour += count_max_q

In [15]:
len(answers)

33387

In [16]:
count_answers_tour

33387

In [17]:
len(dict_players) +  count_answers_tour 

92657

In [39]:
X = sparse.lil_matrix((len(y), len(dict_players) +  count_answers_tour ))

In [40]:
for i, j in idx_where_1_player:
    X[i, j] = 1

In [41]:
for i, j in idx_where_1_answers:
    X[i, j + len(dict_players)] = 1

In [21]:
len(y)

21110047

In [22]:
y = np.array(y)
shape_0, shape_1 = X.shape

In [23]:
baseline_model = LogisticRegression(tol=1e-1, solver="saga", C=10, fit_intercept=False)
baseline_model.fit(X, y)

LogisticRegression(C=10, fit_intercept=False, solver='saga', tol=0.1)

In [24]:
rating = baseline_model.coef_[0][:len(dict_players)]

In [25]:
players_df['rating'] = [0.] * len(players_df)
players_df['count_games'] = [0.] * len(players_df)

In [26]:
for player in dict_players.keys():
    players_df['rating'][player] = rating[dict_players[player]]
    players_df['count_games'][player] = count_answers[player]

<ipython-input-26-1d13074d678b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_df['rating'][player] = rating[dict_players[player]]
<ipython-input-26-1d13074d678b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_df['count_games'][player] = count_anwers[player]


Сделаем фильтр на >100 игр

In [27]:
players_df[players_df['count_games'] > 100].sort_values(by=['rating'], ascending=False)[:20]

,id,name,patronymic,surname,rating,count_games
27403,27403,Максим,Михайлович,Руссо,3.725996,2438.0
28751,28751,Иван,Николаевич,Семушин,3.602872,4082.0
4270,4270,Александра,Владимировна,Брутер,3.569447,2990.0
30270,30270,Сергей,Леонидович,Спешков,3.365261,4154.0
27822,27822,Михаил,Владимирович,Савченков,3.364312,3641.0
30152,30152,Артём,Сергеевич,Сорожкин,3.344988,5218.0
18036,18036,Михаил,Ильич,Левандовский,3.245135,1524.0
20691,20691,Станислав,Григорьевич,Мереминский,3.228953,1703.0
26089,26089,Ирина,Сергеевна,Прокофьева,3.208796,1135.0
22799,22799,Сергей,Игоревич,Николенко,3.193312,2289.0


#### Задание 3.
Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:

- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


***Решение***: Рассмотрим турнир как "один вопрос" и в матрице X - 1 соотвествует игроку команды

In [28]:
tournaments_test.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
4957,4957,Синхрон Биркиркары,2020-02-21 00:00:00+03:00,2020-02-27T23:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 2421, 'name': 'Ася', 'patronymic': 'Се...",{'dateRequestsAllowedTo': '2020-02-27T18:00:00...,"{'1': 13, '2': 13, '3': 13}"
5414,5414,Синхрон северных стран,2020-01-03 19:00:00+03:00,2020-01-10T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2020-01-10T23:59:00...,"{'1': 12, '2': 12, '3': 12}"
5477,5477,Онлайн: Синхрон Урюбджирова,2020-04-18 19:00:00+03:00,2020-04-30T19:00:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 91324, 'name': 'Эрдни', 'patronymic': ...",{'dateRequestsAllowedTo': '2020-04-30T23:55:00...,"{'1': 12, '2': 12, '3': 12}"
5707,5707,Школьный Синхрон-lite. Выпуск 3.6,2020-02-01 00:05:00+03:00,2020-03-09T23:55:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2020-03-06T23:59:00...,"{'1': 12, '2': 12, '3': 12}"
5708,5708,(а)Синхрон-lite. Лига старта. Эпизод XII,2020-02-01 00:05:00+03:00,2020-03-09T23:55:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2020-03-06T23:59:00...,"{'1': 12, '2': 12, '3': 12}"


In [31]:
corr_spearmanr = []
corr_kendalltau = []
for id_tour in tournaments_test['id']:
    count_anw = []
    list_index_players = []
    j = 0
    if results[id_tour] == []:
        continue
    for team in results[id_tour]:
        if not 'mask' in team.keys() or team['mask'] is None:
            continue
        count_anw.append(len(re.findall(r'1',team['mask'])))
        list_append = []
        for playes in team['teamMembers']:
            if playes['player']['id'] in dict_players.keys():
                list_append.append(dict_players[playes['player']['id']])
        list_index_players.append(list_append)
    if len(list_index_players) < 5:
        continue
    X = sparse.lil_matrix((len(list_index_players), shape_1))
    for i in range(len(list_index_players)):
        for j in list_index_players[i]:
            X[i,j] = 1
    
    cat_rating = baseline_model.predict_proba(X)[:, 1]
    corr_spearmanr.append(sts.spearmanr(cat_rating, count_anw).correlation)
    
    corr_kendalltau.append(sts.kendalltau(cat_rating, count_anw).correlation)

In [32]:
print(f'Корреляция Спирмена :{np.mean(corr_spearmanr)}')
print(f'Корреляция Кендалла :{np.mean(corr_kendalltau)}')

Корреляция Спирмена :0.7637880998228315
Корреляция Кендалла :0.6109669327100429


### Задание 4 
Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


***Решение***: 

e-шаги пересчитываем вероятность правильного ответа игрока, как вероятность его ответа /  вероятность, что все в команде ответили правильно

m - шаг: считаем линейную регрессию

In [33]:
def test_metrice(model):
    corr_spearmanr = []
    corr_kendalltau = []
    for id_tour in tournaments_test['id']:
        count_anw = []
        list_index_players = []
        j = 0
        if results[id_tour] == []:
            continue
        for team in results[id_tour]:
            if not 'mask' in team.keys() or team['mask'] is None:
                continue
            count_anw.append(len(re.findall(r'1',team['mask'])))
            list_append = []
            for playes in team['teamMembers']:
                if playes['player']['id'] in dict_players.keys():
                    list_append.append(dict_players[playes['player']['id']])
            list_index_players.append(list_append)
        if len(list_index_players) < 5:
            continue
        
        X_1 = sparse.lil_matrix((len(list_index_players), 92657))
        for i in range(len(list_index_players)):
            for j in list_index_players[i]:
                X_1[i,j] = 1
    
        cat_rating = model.predict_proba(X_1)[:,1]
        corr_spearmanr.append(sts.spearmanr(cat_rating, count_anw).correlation)
    
        corr_kendalltau.append(sts.kendalltau(cat_rating, count_anw).correlation)
  
    print(f'Корреляция Спирмена :{np.mean(corr_spearmanr)}')
    print(f'Корреляция Кендалла :{np.mean(corr_kendalltau)}')

In [45]:
class LinearRegression_probability:
    def __init__(self, init_classifier=None):
        if init_classifier:
            self.w = np.hstack((init_classifier.intercept_, init_classifier.coef_[0]))
        else:
            self.w = None
        
    def fit(self, X, y, max_iters=100000, lr=10, batch_size=1000, tol=0.0000001):
        X = sparse.hstack([np.ones(len(y)).reshape(-1, 1), X], format='csr')
        if self.w is None:
            self.w = np.random.normal(size=X.shape[1])
        losses = []
        prev_mean_loss = np.inf
        data_len = X.shape[0]
        for i in range(max_iters):
            batch_idxs = np.random.choice(data_len, batch_size)
            X_batch = X[batch_idxs, :]
            y_batch = y[batch_idxs]
            preds = expit(X_batch.dot(self.w))
            losses.append(self.log_loss(y_batch, preds))
            if (i + 1) % 1000 == 0:
                new_mean_loss = np.mean(losses)
                if (prev_mean_loss - new_mean_loss) < tol:
                    break                    
                losses = []
                prev_mean_loss = new_mean_loss

            grad = -X_batch.T.dot(y_batch - preds) / len(y_batch)
            self.w -= lr * grad 
    
    def predict_proba(self, X):
        X = sparse.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
        preds = expit(X.dot(self.w))
        return np.hstack(((1 - preds).reshape(-1, 1), preds.reshape(-1, 1)))
    

    def log_loss(y, p):
        return -np.mean(y * np.log(p) + (1 - y) * np.log(1 - p))

In [49]:
model = LinearRegression_probability(init_classifier=baseline_model)

In [47]:
len(teams)

21110047

In [50]:
%%time
for _ in range(5):
    preds = model.predict_proba(X)
    df_players = pd.DataFrame({"team": teams,
                            "answer": answer_list,
                            "fail": preds[:, 0],
                            "succes_player": preds[:, 1]})
    
    df_team = df_players.groupby(["team", "question"])['fail'].prod().reset_index()
    df_team["succes_team"] = 1 - df_team["fail"]
    df_players = pd.merge(df_players, df_team, left_on=['team', "answer"],  right_on=["team", "answer"])
    z = (df_players["succes_player"] / df_players["succes_team"]).clip(0, 1)
    z *= y 
    model.fit(X, z, lr=20)
    test_metrice(model)

Корреляция Спирмена :0.7593418154407581
Корреляция Кендалла :0.6056439272708434
Корреляция Спирмена :0.7679834185465153
Корреляция Кендалла :0.6140835195314843
Корреляция Спирмена :0.7673540734358829
Корреляция Кендалла :0.6138098721218802
Корреляция Спирмена :0.7677257132616863
Корреляция Кендалла :0.6141509575979149
Корреляция Спирмена :0.7693900160762172
Корреляция Кендалла :0.6149531007284342
CPU times: user 12min 44s, sys: 4min 21s, total: 17min 5s
Wall time: 16min 58s


#### Задание 5
А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [52]:
rating_answer = baseline_model.coef_[0][len(dict_players):]
df_answer = pd.DataFrame(answers.values(),columns = ['tour'])
df_answer['rating'] = rating_answer[:len(answers)]
rating_tour = df_answer.groupby('tour')['rating'].mean().reset_index().sort_values(by='rating', ascending=True)

Наиболее сложные:

In [53]:
rating_tour.head(10)

,tour,rating
626,Чемпионат Санкт-Петербурга. Первая лига,-7.509381
551,Угрюмый Ёрш,-3.664632
480,Синхрон высшей лиги Москвы,-3.644532
122,Воображаемый музей,-3.487269
419,Первенство правого полушария,-3.430805
177,Записки охотника,-3.188028
195,Зеркало мемориала памяти Михаила Басса,-3.167253
613,Чемпионат Мира. Этап 2 Группа С,-3.040648
210,Знание – Сила VI,-3.038757
558,Ускользающая сова,-2.998009


 Наиболее легкие:

In [54]:
rating_tour.tail(10)

,tour,rating
646,Школьная лига. II тур.,1.201638
4,(а)Синхрон-lite. Лига старта. Эпизод III,1.226995
9,(а)Синхрон-lite. Лига старта. Эпизод VII,1.235000
530,Студенческий чемпионат Калининградской области,1.271080
647,Школьная лига. III тур.,1.360508
651,Школьный Синхрон-lite. Выпуск 2.5,1.369939
644,Школьная лига,1.390846
645,Школьная лига. I тур.,1.436622
468,Синхрон Лиги Разума,1.495671
7,(а)Синхрон-lite. Лига старта. Эпизод V,1.513908
